<a href="https://colab.research.google.com/github/shubham-ai/ChatGPT/blob/main/Weights_and_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "wandb>=0.15.4" -qqq
!pip install "langchain>=0.0.218" openai -qqq
import langchain
assert langchain.__version__ >= "0.0.218", "Please ensure you are using LangChain v0.0.188 or higher"
import os
!git clone https://github.com/soumik12345/wandb-addons.git
!pip install ./wandb-addons[prompts] openai wandb -qqq
!pip install transformers[torch]
!pip install sentencepiece --user
!pip install einops --user
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "langchain-testing"



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
Cloning into 'wandb-addons'...
remote: Enumerating objects: 1795, done.
remote: Counting objects: 100% (399/399), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 1795 (delta 217), reused 223 (delta 163), pack-reused 1396
Receiving objects: 100% (1795/1795), 5.01 MiB | 16.07 MiB/s, done.
Resolv

In [2]:
from langchain.chains import LLMChain, SimpleSequentialChain
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
import datetime
from wandb_addons.prompts import Trace
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.tools import BaseTool
from typing import Union
from math import pi
from typing import Any
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain, PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import PromptTemplate
from wandb_addons.prompts import Trace
# from wandb.integration.langchain import WandbTracer
import wandb
import datetime
import transformers
import torch
from transformers import AutoTokenizer

In [2]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
!git lfs install
# !git clone https://huggingface.co/google/flan-t5-base
!git clone https://huggingface.co/bigscience/T0pp

Git LFS initialized.
Cloning into 'T0pp'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 134 (delta 77), reused 134 (delta 77), pack-reused 0
Receiving objects: 100% (134/134), 24.17 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (77/77), done.
^C


In [ ]:
wandb.login()

wandb: Currently logged in as: vaibhavarena. Use `wandb login --relogin` to force relogin


True

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain, PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import PromptTemplate
from wandb_addons.prompts import Trace
# from wandb.integration.langchain import WandbTracer
import wandb
import datetime
wandb.init(
    # set the wandb project where this run will be logged
    project="dlai_llm2__",
)

# wandb_config = {"project": "shash36_LLM_19_07_23"}

def fib(n):
    if n <= 1:
        return n
    else:
        return (fib(n-1) + fib(n-2))

memory = ConversationBufferMemory(memory_key="chat_history")

tools = Tool(
            name='Fibonacci', func = lambda n: str(fib(int(n))), description="use when you want to calculate the nth fibonacci number"
            )

model_id = "flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model, tokenizer=tokenizer, max_length=512
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# query = "what's the capital of France?"
# query = "What is verizon?"
# query = "What's the fibonacci of 3?"

queries_ls = [
  "What is the capital of France?",
  "How do I boil an egg?",
  "What to do if the aliens arrive?",
  "How do I boil an egg?*100",
  "What's the fibonacci of 3?",
]

for query in queries_ls:

    try:
        template = """\
        Answer the following query: {query}?
        """
        prompt = PromptTemplate.from_template(template)
        prompt.format(query=query)

        start_time_ms = datetime.datetime.now().timestamp() * 1000
        llm_chain = LLMChain(prompt = prompt,
                            llm=local_llm, verbose=True)

        if len(query)>1000:
            raise ValueError("Cannot proceed")
        else:
            response_text = llm_chain.run(query)
            end_time_ms = round(datetime.datetime.now().timestamp() * 1000)  # logged in milliseconds
            status="success"
            status_message=None

    except ValueError as e:
        end_time_ms = round(datetime.datetime.now().timestamp() * 1000)  # logged in milliseconds
        status="error"
        status_message=str(e)
        response_text = ""

    # create a span in wandb
    root_span = Trace(
          name="root_span",
          kind="chain",  # kind can be "llm", "chain", "agent" or "tool"
          status_code=status,
          status_message=status_message,
          start_time_ms=start_time_ms,
          end_time_ms=end_time_ms,
          inputs={"system_prompt": template, "query": query},
          outputs={"response": response_text},
          )

    # log the span to wandb
    root_span.log(name="openai_trace")

# agent= initialize_agent([tools], local_llm, agent="chat-zero-shot-react-description", verbose=True)
# agent= initialize_agent([tools], local_llm, agent="conversational-react-description", memory=memory, verbose=True)
# agent.run("What's the fibonacci of 3?")
# agent.run("hi")



> Entering new LLMChain chain...
Prompt after formatting:
        Answer the following query: What is the capital of France??
        

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
        Answer the following query: How do I boil an egg??
        

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
        Answer the following query: What to do if the aliens arrive??
        

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
        Answer the following query: How do I boil an egg?*100?
        

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
        Answer the following query: What's the fibonacci of 3??
        

> Finished chain.


# *Testing lagchain with Prompts*

In [8]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp")
# model = AutoModelForSeq2SeqLM.from_pretrained("T0pp")

# inputs = tokenizer.encode("Is this review positive or negative? Review: this is the best cast iron skillet you will ever buy", return_tensors="pt")
# outputs = model.generate(inputs)
# print(tokenizer.decode(outputs[0]))

In [5]:
from langchain.chains import LLMChain, SimpleSequentialChain
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
import datetime
import os

# turn on wandb logging for langchain
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
from wandb.sdk.data_types.trace_tree import Trace
PROJECT = "Langchain"
MODEL_NAME = "flan-t5-base"
wandb.login(anonymous="allow")
run = wandb.init(project=PROJECT, job_type="generation")

model_id = "flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model, tokenizer=tokenizer, max_length=512
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# llm = OpenAI(model_name="text-davinci-003", openai_api_key=API_KEY)

# first step in chain
start_time_ms = datetime.datetime.now().timestamp() * 1000

template = "What is the most popular city in {country} for tourists? Just return the name of the city"

first_prompt = PromptTemplate(

input_variables=["country"],

template=template)

chain_one = LLMChain(llm = local_llm, prompt = first_prompt, verbose=True)

# second step in chain

second_prompt = PromptTemplate(

input_variables=["city"],

template="What are the top three things to do in this: {city} for tourists. Just return the answer as three bullet points.",)

chain_two = LLMChain(llm=local_llm, prompt=second_prompt, verbose=True)

# Third step in chain

thrid_prompt = PromptTemplate(

input_variables=["city"],

template = "What are the top Companies in this: {city}. Just return the answer as three bullet points.",)

chain_three = LLMChain(llm=local_llm, prompt=thrid_prompt, verbose=True)

# fourth step in chain

fourth_prompt = PromptTemplate(

input_variables=["city"],

template = "where is the best place to east food in this {city}",)

chain_four = LLMChain(llm=local_llm, prompt=fourth_prompt, verbose=True)


# Combine the first and the second chain
overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two, chain_three,chain_four], verbose=True)
end_time_ms = round(datetime.datetime.now().timestamp() * 1000)  # logged in milliseconds
query="UK"
final_answer = overall_chain.run(query)

# create a span in wandb
root_span = Trace(
          name="root_span",
          kind="chain",  # kind can be "llm", "chain", "agent" or "tool"
          start_time_ms=start_time_ms,
          end_time_ms=end_time_ms,
          inputs={"query": query},
          outputs={"response": final_answer},
          )
root_span.log(name="openai_trace")
wandb.finish()



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
What is the most popular city in UK for tourists? Just return the name of the city

> Finished chain.
london


> Entering new LLMChain chain...
Prompt after formatting:
What are the top three things to do in this: london for tourists. Just return the answer as three bullet points.

> Finished chain.
London is a great place to visit


> Entering new LLMChain chain...
Prompt after formatting:
What are the top Companies in this: London is a great place to visit. Just return the answer as three bullet points.

> Finished chain.
London Stock Exchange


> Entering new LLMChain chain...
Prompt after formatting:
where is the best place to east food in this London Stock Exchange

> Finished chain.
a burger joint

> Finished chain.


In [ ]:
 # SCENARIO 2 - Chain
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callbacks=callbacks)

test_prompts = [
    {
        "title": "documentary about good video games that push the boundary of game design"
    },
    {"title": "cocaine bear vs heroin wolf"},
    {"title": "the best in class mlops tooling"},
]
synopsis_chain.apply(test_prompts)
wandb_callback.flush_tracker(synopsis_chain, name="agent")

In [8]:
# # !pip install einops --user
# # !pip install sentencepiece --user
# !pip install sentencepiece


# **Testing Lanchin with custom tool Not working**

In [ ]:

from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.tools import BaseTool
from typing import Union
from math import pi
from typing import Any
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

def fib(n):
    if n <= 1:
        return n
    else:
        return (fib(n-1) + fib(n-2))
class AreaofCircle(BaseTool):
    name = "Area of Circle"
    description = "Use this tool Calculate the area of a circle using radius"

    def _run(self, radius : Union[int,float]) -> Any:
        return pi * float(radius) ** 2

    def _async_run(self) -> Any:
        raise NotImplementedError("This tool does not support async run")

# memory = ConversationBufferMemory(memory_key="chat_history")
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    return_messages=True
)
model_id = "flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model, tokenizer=tokenizer, max_length=565
)

local_llm = HuggingFacePipeline(pipeline=pipe)
# tools = Tool(
#             name='Fibonacci', func = lambda n: str(fib(int(n))), description="use when you want to calculate the nth fibonacci number"
#             )
tools = [AreaofCircle()]

# agent= initialize_agent([tools], local_llm, agent='conversational-react-description', memory=memory, verbose=True)
# agent = initialize_agent(
#     agent='chat-zero-shot-react-description',
#     tools=tools,
#     llm=local_llm,
#     verbose=True,
#     memory=conversational_memory,
#     handle_parsing_errors=True,
#     max_iterations=3,
# )
agent= initialize_agent(tools, local_llm, agent="chat-zero-shot-react-description", verbose=True)
# agent= initialize_agent([tools], local_llm, agent="conversational-react-description", memory=memory, verbose=True)
agent.run("What's the fibonacci of 3?")
# agent.run("hi")# agent.run("hi")



> Entering new AgentExecutor chain...


OutputParserException: ignored

https://wandb.ai/vaibhavarena/gpt/reports/Untitled-Report--Vmlldzo1MDc4Njgw

https://api.wandb.ai/links/vaibhavarena/bueecu39

# **Simple Generations**

In [ ]:
import os
import random
import time
import datetime

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)
import wandb
from wandb.sdk.data_types.trace_tree import Trace

In [ ]:
PROJECT = "dlai_llm"
MODEL_NAME = "gpt-3.5-turbo"
run = wandb.init(project=PROJECT, job_type="generation")

In [ ]:
# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
model_id = "flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model, tokenizer=tokenizer, max_length=1024
)

local_llm = HuggingFacePipeline(pipeline=pipe)
def completion_with_backoff(query):
    template = """\
    Answer the following story: {query}?
    """
    prompt = PromptTemplate.from_template(template)
    prompt.format(query=query)

    start_time_ms = datetime.datetime.now().timestamp() * 1000
    llm_chain = LLMChain(prompt = prompt,
    llm=local_llm, verbose=True)
    if len(query)>1000:
            raise ValueError("Cannot proceed")
    else:
            response_text = llm_chain.run(query)
            end_time_ms = round(datetime.datetime.now().timestamp() * 1000)  # logged in milliseconds
            status="success"
            status_message=None
            return response_text
stem_prompt = """You are a creative copywriter.
You're given a category of game asset, \
and your goal is to design a name of that asset.
The game is set in a fantasy world \
where everyone laughs and respects each other,
while celebrating diversity."""

completion_with_backoff(query=stem_prompt)



> Entering new LLMChain chain...
Prompt after formatting:
    Answer the following story: You are a creative copywriter.
You're given a category of game asset, and your goal is to design a name of that asset.
The game is set in a fantasy world where everyone laughs and respects each other, 
while celebrating diversity.?
    

> Finished chain.


'samurai'